In [ ]:
!pip install transformers

In [ ]:
system_prompt = """You are a clinical assistant that writes concise, structured summaries of dialogues between a health worker and a patient.
given a dialogues between Health Worker and Patient related Head and Neck cancer and Blood donations. You are provided the dialogues in language **{}**. Strictly Provide Output in only English. No other language.

Generate a textual summary with free-flow headings and subheadings.
###  RULES
Important rule - add relevant summarization heading
1. Do include heading and subheadings
2. Stictly Do not repeat the same headings.
3. **Output language** English only, even if the dialogue is in another language.
4. **Faithfulness:** Use only facts mentioned in the dialogue. Do NOT infer, assume, or add new medical details.
5. **Style:** Professional and neutral; avoid conversational tone or filler phrases.
6. Provide a clear and detailed summaries with headings and bullet points under each heading/sub-heading.
7. Based on the discussions, you provide information through different Headings and Sub headings. The Summary Structure is based on Patient and Health worker discussions.


- Setting and patient context
  - Should have Details about patient’s age, name, location, who accompanied.

- Presenting symptoms and onset
  - Should have Main complaints, how long they have been present, nature of onset

- Risk factors and counseling
  - Should have Risk factors Tobacco, alcohol, diet, HPV, lifestyle guidance, any preventive counseling given

- Diagnostic framing and planning
  - Should contain details like Clinical suspicion, reasoning, age-related or risk-related framing, next diagnostic steps

- Diagnostic plan and components
  - Should contain Diagnostic Plan and Components such as Tests, imaging, timelines, rationale for each test

- Documentation and preparation
  - Should have Documents required, administrative steps, support from relatives

- HPV information and follow-up
  - Should have Education, vaccination, awareness topics covered

- Current status and forward plan
  - Present state, what’s pending, next visits or planned investigations

- Treatment considerations (if cancer is suspected or diagnosed)
  - Possible treatment options discussed, staging relevance, timelines

- Red-flag symptoms requiring urgent evaluation
  - List of key symptoms that would trigger urgent review

- Logistics, scheduling, and family involvement
  - Test scheduling, duration, need for family support

- Overall takeaway
  - Concise final summary in 3-4 lines which contains goals, next steps, and education provided



### OUTPUT FORMAT
*** Output Should be strictly in English*** Do not include any other languages.
Return only plain text (no markdown, no JSON).
Do NOT include examples or reuse any earlier summaries.
You are free to use different Headings and Sub Headings and corresponding information based on the information in the dialogue

"""


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "vinaybabu/Qwen3-1.7B-SharedTask-Finetuned-Summarization_V2"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
## Mount drive
## Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = "/content/drive/MyDrive/nlpai4health-shared-task/test_data_release"   # add test dataset location

In [ ]:
import os
list_of_files = ['Hindi']    # select the language folder - I have run for each language seperately due to timeout issues

In [ ]:
i=0
output_folder_to_write = "/content/drive/MyDrive/nlpai4health-shared-task/test_data_release/Hindi/Summary_Text_Output/"  # Mention folder to write the output
list_of_files_in_output = os.listdir(output_folder_to_write)
list_of_files_in_output_with_acutal_names = extracted = [f.split("Open_")[-1].replace(".txt", "") for f in list_of_files_in_output]
for language in list_of_files:
  dialogue_folder = "Dialogues"
  list_of_files_folder1 = os.listdir(os.path.join(dataset_path, language, dialogue_folder))
  print(list_of_files_folder1)
  for file_read in list_of_files_folder1:
    if file_read.split(".")[0] in list_of_files_in_output_with_acutal_names:
      print("file already exist {}",file_read)
      continue
    else:
      with open(os.path.join(dataset_path, language, dialogue_folder, file_read), "r", encoding="utf-8") as f:
        dialogue = f.read()
      messages = [{"role":"system","content":system_prompt},{"role":"user","content":dialogue}]
      print(i)
      i = i+1
      text = tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True,enable_thinking=False) # Switches between thinking and non-thinking modes. Default is True.)
      model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
            # conduct text completion
      generated_ids = model.generate(**model_inputs,max_new_tokens=32768)
      output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
        # parsing thinking content
      try:
          # rindex finding 151668 (</think>)
          index = len(output_ids) - output_ids[::-1].index(151668)
      except ValueError:
          index = 0

      thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
      content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
      print(content)
      print("writing_to_file")
      file_to_write = "Team_KV"+"_Summarization"+"_Closed_"+file_read.split(".")[0]+".txt"
      with open(output_folder_to_write+file_to_write, "w") as file:
        file.write(content)

['scenario_8_e149fa6f736447b2b721048b6981f44c_IDX_01_2.jsonl', 'scenario_7_8f1ae76653384eda919bbb4a2f9f57aa_IDX_02_4.jsonl', 'scenario_12_1a7ae83a557e4e9e88a4d67062300e36_IDX_03_3.jsonl', 'scenario_40_e5c75395f52f4033a204772b0fff6112_IDX_02_1.jsonl', 'scenario_32_175cc6577b5a4e9db0cd4a6fc0888a62_IDX_00_2.jsonl', 'scenario_1_b3104a777c094f7abd1b2db26b024efc_IDX_05_1.jsonl', 'scenario_14_011e5beec17c4141a6fc06f44ca23593_IDX_05_2.jsonl', 'scenario_12_021292705e3d4da892d8b08dc525a14d_IDX_04_4.jsonl', 'scenario_33_fff53212697c4bc5a0728e23fd70beda_IDX_05_2.jsonl', 'scenario_6_d6eb63c4146d44dc92c4ccdcb300e8b2_IDX_01_4.jsonl', 'scenario_11_07546481e9eb4e97985d5bb5211c807e_IDX_05_4.jsonl', 'scenario_0_c0d64a81449947ab84e26928c1859665_IDX_03_2.jsonl', 'scenario_42_6a83eeeecf8649399a2cbff370347a92_IDX_02_1.jsonl', 'scenario_29_72f4d2cd165241fcbac762333623290c_IDX_04_3.jsonl', 'scenario_7_fb5d3b42490544f38b6b7c1ae4bb98f3_IDX_04_0.jsonl', 'scenario_37_b126f64ecc864fcfbde01100e882ec9a_IDX_00_0.jsonl

This is a friendly reminder - the current text generation call has exceeded the model's predefined maximum length (40960). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Streaming output truncated to the last 5000 lines.

**Diagnostic framing and planning**
- **Diagnostic framing**: Sweat test is the primary test for CF; baseline data (weight, stool diary, feeding log) will be used to assess risk.
- **Plan**: Sweat test scheduled; baseline data collected; genetic counseling planned for family.

**Diagnostic plan and components**
- **Diagnostic plan**: Sweat test (if feasible); baseline data (weight, stool diary, feeding log) collected; genetic counseling for family.
- **Components**: Sweat test; baseline data collection; genetic counseling.

**Documentation and preparation**
- **Documents required**: ID, photograph, birth certificate, weight chart, stool diary, feeding log, travel plan.
- **Preparation**: Travel logistics and tele-support planning.

**HPV information and follow-up**
- **HPV information**: HPV vaccination information provided; follow-up plan explained.
- **Follow-up**: HPV vaccine status noted; follow-up plan explained.

**Current statu